## Automation - Scraping data publications from NHS England web pages and consolidating them into one file.

This demonstration will show you how you can automatically download multiple published data files from different (related) web pages and consolidate them into one file. The example is of the **Learning Disability Health Check Scheme**

[Data landing page](https://digital.nhs.uk/data-and-information/publications/statistical/learning-disabilities-health-check-scheme)

### Beautiful Soup

[Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) is a popular webscraping library for Python. It can be used to extract data from HTMl and XML files, most germanely from websites. This can include extracting data from flat tables that appear on the web pages themselves (which will be covered in a future Code Club session), but here we will be using it to access .csv files hosted on a series of related web pages.

### Using `urljoin` to construct URLs

The code block below will extract any .csv files for the calendar year 2024, which are all saved to their individual web pages, meaning that urljoin is required to construct the URLs dynamically (i.e. so that you don't have to hard code all the indidual web pages).

The package "re" is imported so that regular expression logic can be used in the construction of the URLs i.e. anything matching the patterm of the regular expression will be considered a web page of interest. (NOTE: you do not need to install "re", it is native to Python)

It's been limited to 2024 files to reduce the amount of data being transferred, but you could use a different regular expression to cover more.

### Re-usability

The code below may look a little scary, but the great news is that you can re-use it for your work without much modification at all! All you need to do is change the URL to the address of the base web page (the one that acts as the landing page for the separate data files) and update the "dynamic_section" to a regular expression that matches the pattern for the sub-URLs hosting each of the data files.

You can find plenty of examples of regular expression patterns [here](https://www.programiz.com/python-programming/regex)

In [ ]:
from urllib.parse import urljoin
import requests as req
import re
import os
from bs4 import BeautifulSoup

url = 'https://digital.nhs.uk/data-and-information/publications/statistical/learning-disabilities-health-check-scheme'

target_urls = []                           # empty list that will later get filled with target URLs in a for loop.

dynamic_section = r'^england-[a-z]+-2024$' # the regular expression for the URLs we are interested in. note that the $ implies that you don't want anything else to follow.

response = req.get(url)                  # get the response from the base URL

if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")     # if there is a successful response, create a BeautifulSoup object.

    for link in soup.find_all('a', href = True):
        sublink = link["href"]
        if re.match(dynamic_section,sublink.split('/')[-1]):
            full_url = urljoin(url, sublink)                   # for each of the instances of the pattern we are looking for
            target_urls.append(full_url)                        # add the constructed full URL to a list of target URLs
        
    for link in target_urls:                                    # check for a successful response (code 200) from each URL
        response = req.get(link)                                # and create a BeautifulSoup object for each.
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")

            for link in soup.find_all("a", href=True):          # for each URL found on each of the pages in target_urls...
                file_url = link['href']                         

                if file_url.endswith(('.csv')):                 # ... check for .csv file extensions
                    print("Found .csv file:", file_url)

                    file_name = file_url.split("/")[-1]         # extract the file name from the URL i.e. everything after the last /
                    file_response = req.get(file_url)           # check the response for each file

                    file_path = os.path.join("downloads",       # set up the file path for the downloaded files to the "downloads" folder.
                                              file_name)
            
                    if file_response.status_code == 200:        # if there's a successful response
                        
                        with open(file_path, "wb") as file:     # save the file to the downloads directory
                            file.write(file_response.content)
                        print(f"Downloaded: {file_name}")
                    else:
                        print(f"Failed to download: {file_url}")

else:
    print(f'Failed to fetch webpage: {response.status_code}')   # this else statement pairs with the original response code check for the base URL
                                                                # (see the first "if" in this code block)

### Consolidating the files into one

Downloading all the files is one thing, but if we want to work with a complete dataset covering a specific time period, we want the data consolidated into one file. The code block below will do this for us. The benefit of using code like this is that it will ensure that no file is left untouched and no file gets added to the consolidated dataset twice. Have you ever had to do this manually and have lost track as to whether you have already added a file's data to the master copy, and then missed or duplicated a month?

In [ ]:
import os # for performing operating system operations, such as accessing file directories

import glob  # used for finding files matching a particular pattern

import pandas as pd # for working with dataframes

input_path = './downloads'

output_path = './final_output'

file_list = glob.glob(os.path.join(input_path, '*.csv')) # get all the .csv files in the downloads folder

# a little function to extract the data from each .csv file, which we will put into a loop
def extract_data(file):
    extract = pd.read_csv(file)
    return extract

# create a single data extract by concatenating the contents of each file in the downloads folder
# "for file in file_list" is our loop i.e. extract data from the file for every file in our list of files
data_extract = pd.concat([extract_data(file) for file in file_list], ignore_index=True)

# export the consolidated data to a single .csv file
data_extract.to_csv(f'{output_path}/consolidated_data.csv', index=False)
